Import libraries and dependencies

In [43]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta

Parse the site for the calendar

In [44]:
url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

Extract the data

In [45]:
rows = soup.find_all("tr")
dates = []
dows = []
texts = []

def process_date_range(date_range_str, text):
    date_range = date_range_str.split("-")
    start_raw = date_range[0].strip() + ", 2021"
    month = start_raw.split(" ")[0]
    end_raw = month + " " + date_range[1].strip() + ", 2021"
    
    # Convert to datetime objects
    start_obj = datetime.strptime(start_raw, "%B %d, %Y").date()
    end_obj = datetime.strptime(end_raw, "%B %d, %Y").date()
    
    # Loop through range and append each date
    date_list = []
    while start_obj <= end_obj:
        date_list.append((start_obj.strftime("%Y-%m-%d"), start_obj.strftime("%A"), text))
        start_obj += timedelta(days=1)
    return date_list

for row in rows:
    cells = row.find_all("td")
    if len(cells) >= 3: 
        date = cells[0].text.strip()
        dow = cells[1].text.strip()  
        text = cells[2].text.strip()
        
        text = text.replace("\n", " ").replace("\t", " ").strip()
        
        # Date ranges
        if "-" in date:
            date_list = process_date_range(date, text)
            for formatted_date, dow, text in date_list:
                dates.append(formatted_date)
                dows.append(dow)
                texts.append(text)
        else:
            # Single dates
            if "2022" in date:
                date_obj = datetime.strptime(date, "%B %d, %Y").date()
            else:
                date += ", 2021"
                date_obj = datetime.strptime(date, "%B %d, %Y").date()
                
            dates.append(date_obj.strftime("%Y-%m-%d"))
            dows.append(date_obj.strftime("%A"))
            texts.append(text)

Build the dataframe

In [46]:
df = pd.DataFrame({'date': dates, 'dow': dows, 'text': texts})
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

print(df)

                  dow                                               text
date                                                                    
2021-08-01     Sunday  Application for degree for January and Februar...
2021-08-18  Wednesday                 Last day to apply for Study Abroad
2021-08-24    Tuesday  Last day of Registration;    Last day to file ...
2021-08-25  Wednesday  Start of Fall Term;    Classes begin;    Initi...
2021-08-25  Wednesday          Change of program period; late fees apply
2021-08-26   Thursday          Change of program period; late fees apply
2021-08-27     Friday          Change of program period; late fees apply
2021-08-28   Saturday          Change of program period; late fees apply
2021-08-29     Sunday          Change of program period; late fees apply
2021-08-30     Monday          Change of program period; late fees apply
2021-08-31    Tuesday          Change of program period; late fees apply
2021-08-26   Thursday                     Last day 